# DIVISIÓN DEL CONJUNTO DE DATOS

## Concepto básico de división de datos

La división de un conjunto de datos en **train**, **validation** y **test** es fundamental en el aprendizaje automático para:

 *   Entrenar el modelo con datos de entrenamiento.

  *  Ajustar hiperparámetros y prevenir sobreajuste usando datos de validación.

  *  Evaluar el desempeño final del modelo con datos de prueba que el modelo no ha visto antes.


## División típica y razones

1) **El conjunto de prueba existe (o existirá en el futuro cercano) pero no está disponible por el momento.** La proporción ideal es 3:1. Esto significa que un 75 % es para el conjunto de entrenamiento y un 25 % es para el conjunto de validación. Este escenario se usará en nuestra plataforma de entrenamiento en línea.

2) **El conjunto de prueba no existe.** En ese caso los datos fuente deben dividirse en tres partes: entrenamiento, validación y prueba. Usualmente, el tamaño del conjunto de validación y del de prueba son iguales. Esto da como resultado una proporción de datos fuente de 3:1:1


*    **Train:** 60-80% de los datos para entrenar.

*   **Validation:** 10-20% para ajustar parámetros (no siempre se usa, a veces se usa validación cruzada).

*    **Test:** 10-20% para evaluación final independiente.


## Métodos de división

*    División aleatoria con la función `train_test_split` de `Scikit-learn`.

*    División manual con `Pandas`, útil para datos temporales o donde el orden importa.

*    Estratificación para asegurar que la distribución de clases sea similar en cada subconjunto.


## Explicación de los parámetros

    train_test_split(*arrays, test_size=None, train_size=None, 
        random_state=None, shuffle=True, stratify=None)

*    `*arrays`: Secuencia de arreglos o dataframes (X, y, etc.) que se quieren dividir. Deben tener la misma longitud o número de filas.

*    `test_size`: Proporción (valor float entre 0 y 1) o número entero del conjunto de datos que se reservará para el conjunto de prueba (test). Por ejemplo, `0.2` significa reservar el 20% para test. Si es `None`, se asigna automáticamente según `train_size` o el valor por defecto (`0.25`).

*    `train_size`: Proporción o número entero del conjunto de datos que se reservará para el conjunto de entrenamiento (train). Si es `None`, se usa el complemento del tamaño del `test_size`.

*    `random_state`: Número entero para fijar la semilla del generador aleatorio y así obtener divisiones reproducibles entre ejecuciones.

*    `shuffle`: Booleano (`True` o `False`) que indica si se deben mezclar los datos antes de dividir. Por defecto es `True` para evitar sesgos, pero se puede poner `False` en data temporales.

*    `stratify`: Array que indica la variable con la que se quiere estratificar la división. Normalmente se usa la variable objetivo (y) para asegurar que la proporción de clases quede igual en train y test.

## Ejemplo paso a paso

Supongamos un DataFrame df con variables predictoras y una variable objetivo (target) y.

### Paso 1: Separar características y objetivo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Datos de ejemplo
df = pd.DataFrame({
    'feature1': range(100),
    'feature2': range(100, 200),
    'target': [0]*50 + [1]*50
})

X = df.drop('target', axis=1)
y = df['target']

### Paso 2: Dividir en training+validation y test

In [2]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
    )

Aquí separamos el 20% para test, usando stratify=y para mantener la proporción de clases.

### Paso 3: Dividir training+validation en training y validation

In [3]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)

Aquí se toma el 25% del 80% inicial (que es 20% total) como validación.

### Paso 4: Confirmar tamaños

In [4]:
print(f"Train size: {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size: {len(X_test)}")

Train size: 60
Validation size: 20
Test size: 20


## División manual para datos temporales

Cuando el orden importa (por ejemplo, series temporales), no se recomienda separar aleatoriamente. En su lugar, se usa la división secuencial:

In [5]:
p_train = 0.7
p_val = 0.15

train = df[:int(len(df) * p_train)] # [:70]
val = df[int(len(df) * p_train) : int(len(df) * (p_train + p_val))] # [70:85]
test = df[int(len(df) * (p_train + p_val)):] # [85:]

print(f"Train size: {len(train)}")
print(f"Validation size: {len(val)}")
print(f"Test size: {len(test)}")

Train size: 70
Validation size: 15
Test size: 15


## Detalles importantes y consejos

*    Fijar random_state para reproducibilidad.

*    Usar stratify si la variable objetivo es categórica para evitar desbalance.

*    Validación cruzada puede sustituir validación si se prefiere.

*    Asegurarse que ningún dato esté duplicado entre sets.
